In [156]:
# Import Libraries
import pandas as pd 
from glob import glob
import os
import mysql.connector as connect

## ETL functions

In [157]:
def extract_data(stock_files):
    '''Extract the data from folder and concatenating all csv file in pandas dataframe'''
    data = pd.concat((pd.read_csv(file,encoding='cp1252')
          for file in stock_files), ignore_index = True)
    return data


def tranfrom_data(data): 
    '''Transform dataset - Drop columns, Replace NaN value'''
    
    if (("Unnamed: 10" in data) == True) and (("Unnamed: 9" in data) == True) and (("ï»¿report_date" in data) == True):
        data['report_date'].fillna(data['ï»¿report_date'], inplace=True)
        data = data.drop(['Unnamed: 10','Unnamed: 9','ï»¿report_date'], axis = 1)

    elif ("Unnamed: 9" in data) == True and ("ï»¿report_date" in data) == True:
        data['report_date'].fillna(data['ï»¿report_date'], inplace=True)
        data = data.drop(['Unnamed: 9','ï»¿report_date'], axis = 1)

    elif ("ï»¿report_date" in data) == True:
        data['report_date'].fillna(data['ï»¿report_date'], inplace=True)
        data = data.drop(['ï»¿report_date'], axis = 1)

    elif ("Unnamed: 9" in data) == True:
        data = data.drop(['Unnamed: 9'], axis = 1)
    
    # drop unnecessary columns
    data = data.drop(['data_field_code','time_period','time_period_type','unit'], axis = 1)
    # replace empty value with 'NaN'
    data = data.fillna('NAN')
    # rename col
    data = data.rename(columns = {'value':'cases'})
    
    return data

def load_data(data,country_name):
    '''Load dataset into mysql database'''
    # connect to the database
    db = connect.connect(host="localhost",user="root",password="root",database="zikadataset")

    mycursor = db.cursor()
    mycursor.execute('use zikadataset')

    # create table of different country
    mycursor.execute(f"""create table {country_name}(report_date TEXT,
                     location TEXT,
                     location_type TEXT,
                     data_field TEXT,
                     cases TEXT)""")


    # creating column list for insertion
    cols = "`,`".join([str(i) for i in data.columns.tolist()])

    # Insert DataFrame recrds one by one.
    for i,row in data.iterrows():
        sql = f"INSERT INTO `{country_name}` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
        mycursor.execute(sql, tuple(row))

        # the connection is not autocommitted by default, so we must commit to save our changes
        db.commit()
    
    return f'{country_name} Data loaded successfully'

In [160]:
def get_data():
    dirList = os.listdir('data')
    for i in dirList:
        print(i)
        stock_files = sorted(glob(f'data/{i}/*.csv'))
        df = extract_data(stock_files)
        df = tranfrom_data(df)
        load_data(df,i)
        print(f'{i} data loaded')
        
    return 'getting data...'



Argentina
Argentina data loaded
Haiti
Haiti data loaded
Nicaragua
Nicaragua data loaded
Puerto_Rico
Puerto_Rico data loaded
successful


In [164]:
# load data from database
db = connect.connect(host="localhost",user="root",password="root",database="zikadataset")

mycursor = db.cursor()

mycursor.execute("SELECT * FROM argentina")

myresult = mycursor.fetchall()

for x in myresult:
    print(x)
    

('2017-01-12', 'Argentina-Buenos_Aires', 'province', 'cumulative_confirmed_imported_cases', '0.0')
('2017-01-12', 'Argentina-CABA', 'province', 'cumulative_confirmed_imported_cases', '1.0')
('2017-01-12', 'Argentina-Cordoba', 'province', 'cumulative_confirmed_imported_cases', '2.0')
('2017-01-12', 'Argentina-Entre_Rios', 'province', 'cumulative_confirmed_imported_cases', '0.0')
('2017-01-12', 'Argentina-Santa_Fe', 'province', 'cumulative_confirmed_imported_cases', '2.0')
('2017-01-12', 'Argentina-Mendoza', 'province', 'cumulative_confirmed_imported_cases', '1.0')
('2017-01-12', 'Argentina-San_Juan', 'province', 'cumulative_confirmed_imported_cases', '0.0')
('2017-01-12', 'Argentina-San_Luis', 'province', 'cumulative_confirmed_imported_cases', '0.0')
('2017-01-12', 'Argentina-Chaco', 'province', 'cumulative_confirmed_imported_cases', '0.0')
('2017-01-12', 'Argentina-Corrientes', 'province', 'cumulative_confirmed_imported_cases', '0.0')
('2017-01-12', 'Argentina-Formosa', 'province', 'cu

('2016-04-16', 'Argentina-Corrientes', 'province', 'cumulative_probable_imported_cases', '0.0')
('2016-04-16', 'Argentina-Formosa', 'province', 'cumulative_probable_imported_cases', '0.0')
('2016-04-16', 'Argentina-Misiones', 'province', 'cumulative_probable_imported_cases', '0.0')
('2016-04-16', 'Argentina-Catamarca', 'province', 'cumulative_probable_imported_cases', '0.0')
('2016-04-16', 'Argentina-Jujuy', 'province', 'cumulative_probable_imported_cases', '0.0')
('2016-04-16', 'Argentina-Salta', 'province', 'cumulative_probable_imported_cases', '0.0')
('2016-04-16', 'Argentina-Sgo_Del_Estero', 'province', 'cumulative_probable_imported_cases', '0.0')
('2016-04-16', 'Argentina-Tucuman', 'province', 'cumulative_probable_imported_cases', '0.0')
('2016-04-16', 'Argentina-La_Rioja', 'province', 'cumulative_probable_imported_cases', '0.0')
('2016-04-16', 'Argentina-San_Luis', 'province', 'cumulative_probable_imported_cases', '0.0')
('2016-04-16', 'Argentina-Mendoza', 'province', 'cumulative

('2016-07-16', 'Argentina-Sgo_Del_Estero', 'province', 'cumulative_confirmed_local_cases', '0.0')
('2016-07-16', 'Argentina-Tucuman', 'province', 'cumulative_confirmed_local_cases', '22.0')
('2016-07-16', 'Argentina-La_Rioja', 'province', 'cumulative_confirmed_local_cases', '0.0')
('2016-07-16', 'Argentina-San_Luis', 'province', 'cumulative_confirmed_local_cases', '0.0')
('2016-07-16', 'Argentina-Mendoza', 'province', 'cumulative_confirmed_local_cases', '0.0')
('2016-07-16', 'Argentina-San_Juan', 'province', 'cumulative_confirmed_local_cases', '0.0')
('2016-07-16', 'Argentina-Chubut', 'province', 'cumulative_confirmed_local_cases', '0.0')
('2016-07-16', 'Argentina-La_Pampa', 'province', 'cumulative_confirmed_local_cases', '0.0')
('2016-07-16', 'Argentina-Rio_Negro', 'province', 'cumulative_confirmed_local_cases', '0.0')
('2016-07-16', 'Argentina-Santa_Cruz', 'province', 'cumulative_confirmed_local_cases', '0.0')
('2016-07-16', 'Argentina-Neuquen', 'province', 'cumulative_confirmed_loca

('2016-11-02', 'Argentina-San_Juan', 'province', 'cumulative_confirmed_zika_pregnant', '0.0')
('2016-11-02', 'Argentina-San_Luis', 'province', 'cumulative_confirmed_zika_pregnant', '0.0')
('2016-11-02', 'Argentina-Chaco', 'province', 'cumulative_confirmed_zika_pregnant', '0.0')
('2016-11-02', 'Argentina-Misiones', 'province', 'cumulative_confirmed_zika_pregnant', '0.0')
('2016-11-02', 'Argentina-Catamarca', 'province', 'cumulative_confirmed_zika_pregnant', '0.0')
('2016-11-02', 'Argentina-Jujuy', 'province', 'cumulative_confirmed_zika_pregnant', '0.0')
('2016-11-02', 'Argentina-Salta', 'province', 'cumulative_confirmed_zika_pregnant', '0.0')
('2016-11-02', 'Argentina-Tucuman', 'province', 'cumulative_confirmed_zika_pregnant', '4.0')
('2016-11-02', 'Argentina-Neuquen', 'province', 'cumulative_confirmed_zika_pregnant', '0.0')
('2016-11-02', 'Argentina-Buenos_Aires', 'province', 'cumulative_possible_zika_pregnant', '0.0')
('2016-11-02', 'Argentina-CABA', 'province', 'cumulative_possible_z

## Count number of rows and cols

In [35]:
def extract_data(stock_files):
    '''Extract the data from folder and concatenating all csv file in pandas dataframe'''
    data = pd.concat((pd.read_csv(file,encoding='cp1252')
          for file in stock_files), ignore_index = True)
    return data

country = {}

# find all dataset folders
dirList = os.listdir('Dataset')
print(dirList)

for i in dirList:
    # Iterate each folder and merge all csv file of that folder
    stock_files = sorted(glob(f'Dataset/{i}/*.csv'))
    
    # pass all csv to extract_data function
    df = extract_data(stock_files)
    
    # Count total number of rows and cols
    rows_and_cols = {i:[df.shape[0],df.shape[1]]}
    country.update(rows_and_cols)
    
print()
print(country)

['Argentina', 'Brazil', 'Colombia', 'Dominican_Republic', 'Ecuador', 'El_Salvador', 'Guatemala', 'Haiti', 'Mexico', 'Nicaragua', 'Panama', 'Puerto_Rico', 'United_States', 'US_Virgin_Islands']

{'Argentina': [5788, 9], 'Brazil': [9559, 9], 'Colombia': [158460, 9], 'Dominican_Republic': [28496, 9], 'Ecuador': [2943, 11], 'El_Salvador': [4160, 12], 'Guatemala': [4253, 10], 'Haiti': [52, 9], 'Mexico': [12960, 9], 'Nicaragua': [207, 9], 'Panama': [6445, 11], 'Puerto_Rico': [1118, 10], 'United_States': [5869, 9], 'US_Virgin_Islands': [2140, 10]}


In [36]:
new_df = pd.DataFrame.from_dict(country, orient ='index', columns=['Rows','Columns'])
new_df

,Rows,Columns
Argentina,5788,9
Brazil,9559,9
Colombia,158460,9
Dominican_Republic,28496,9
Ecuador,2943,11
El_Salvador,4160,12
Guatemala,4253,10
Haiti,52,9
Mexico,12960,9
Nicaragua,207,9


## Checking all the dataset

In [152]:
def extract_data(stock_files):
    '''Extract the data from folder and concatenating all csv file in pandas dataframe'''
    data = pd.concat((pd.read_csv(file,encoding='cp1252')
          for file in stock_files), ignore_index = True)
    return data

def tranfrom_data(data):  
    if (("Unnamed: 10" in data) == True) and (("Unnamed: 9" in data) == True) and (("ï»¿report_date" in data) == True):
        data['report_date'].fillna(data['ï»¿report_date'], inplace=True)
        data = data.drop(['Unnamed: 10','Unnamed: 9','ï»¿report_date'], axis = 1)

    elif ("Unnamed: 9" in data) == True and ("ï»¿report_date" in data) == True:
        data['report_date'].fillna(data['ï»¿report_date'], inplace=True)
        data = data.drop(['Unnamed: 9','ï»¿report_date'], axis = 1)

    elif ("ï»¿report_date" in data) == True:
        data['report_date'].fillna(data['ï»¿report_date'], inplace=True)
        data = data.drop(['ï»¿report_date'], axis = 1)

    elif ("Unnamed: 9" in data) == True:
        data = data.drop(['Unnamed: 9'], axis = 1)
        
    data = data.drop(['data_field_code','time_period','time_period_type','unit'], axis = 1)
    data = data.fillna('NAN')
    data = data.rename(columns = {'value':'cases'})
    
    return data

# find all dataset folders
dirList = os.listdir('Dataset')
print(dirList)

# Iterate each folder and merge all csv file of that folder
stock_files = sorted(glob(f'Dataset/{dirList[1]}/*.csv'))
 
# pass all csv to extract_data function
data = extract_data(stock_files)
data = tranfrom_data(data)
        

data

['Argentina', 'Brazil', 'Colombia', 'Dominican_Republic', 'Ecuador', 'El_Salvador', 'Guatemala', 'Haiti', 'Mexico', 'Nicaragua', 'Panama', 'Puerto_Rico', 'United_States', 'US_Virgin_Islands']


,report_date,location,location_type,data_field,cases
0,2016-02-13,Brazil-Acre,state,microcephaly_under_investigation,22.0
1,2016-02-13,Brazil-Alagoas,state,microcephaly_under_investigation,90.0
2,2016-02-13,Brazil-Amapa,state,microcephaly_under_investigation,NAN
3,2016-02-13,Brazil-Amazonas,state,microcephaly_under_investigation,NAN
4,2016-02-13,Brazil-Bahia,state,microcephaly_under_investigation,583.0
...,...,...,...,...,...
9554,2016-06-11,Brazil-Mato_Grosso_do_Sul,state,zika_reported,762.0
9555,2016-06-11,Brazil-Mato_Grosso,state,zika_reported,19985.0
9556,2016-06-11,Brazil-Goias,state,zika_reported,4132.0
9557,2016-06-11,Brazil-Distrito_Federal,state,zika_reported,367.0
